In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [13]:
import torch
import json
import pandas as pd
import numpy as np
import nltk
import re
from collections import defaultdict

BiLSTM(Q)
BiLSTM(C)
softmax([Ocs, Oce]WOq)
cross_entropy

In [48]:
def bioclean(t):
    return ' '.join(re.sub('[.,?;*!%^&_+():-\[\]{}]', '',
                        (t.replace('"', '').replace('/', '')
                         .replace('\\', '').replace("'", '')
                         .strip().lower()))
                    .split())
def getQAPair(data):
    qa_pair = []
    count = 0
    for obj in data['questions']:
        if obj['type'] in ['factoid']:
            q = obj['body']
            a = obj['exact_answer']
            if obj['type'] == 'factoid' and isinstance(a[0], list):
                assert len(a) == 1 
                a = a[0]
            t = obj['type']
            s = []
            for snippet in obj['snippets']:
                if snippet['text'] not in s:
                    s.append(snippet['text'])
                else:
                    count += 1
            qa_pair.append([q, a, t, s])
    print ('duplicate snippet count:', count)
    return qa_pair

def getWordDict(qa_pair_clean):
    dct = set()
    for (q, a, t, s) in qa_pair_clean:
        for word in q:
            dct.add(word)
        for ans in a:
            for word in ans:
                dct.add(word)
        for snippet in s:
            for word in snippet:
                dct.add(word)
    return dct

def getCleanQAPair(qa_pair):
    qa_pair_clean = []
    for (q, a, t, s) in qa_pair:
        q = bioclean(q).split()
        a = [bioclean(x).split() for x in a]
        t = t
        s = [bioclean(x).split() for x in s]
        qa_pair_clean.append([q, a, t, s])
    return qa_pair_clean

def loadEmbedBioASQ(w_dct, vec_path, type_path):
    linenum2word = {}
    with open(type_path, encoding='utf8') as f:
        for i, line in enumerate(f):
            w = line.strip('\n')
            if w in w_dct:
                linenum2word[i] = w
    print (len(linenum2word), 'words found for all', len(w_dct), 'words')
    word_vec = []
    word2ind = defaultdict(lambda:len(word2ind))
    with open(vec_path) as f:
        for i, line in enumerate(f):
            if i in linenum2word:
                word = linenum2word[i]
                vec = np.array([float(x) for x in line.strip('\n').split()])
                word_vec.append(vec)
                word2ind[word] 
    return (np.array(word_vec), word2ind)

def loadEmbedGloVe(w_dct, vec_path):
    word2ind = defaultdict(lambda:len(word2ind))
    word_vec = []

    with open(vec_path, encoding='utf-8') as f:
        for i, line in enumerate(f):
            line = line.strip('\n').split(' ')
            word = bioclean(line[0])
            if word in w_dct and word not in word2ind:
                vec = np.array([float(x) for x in line[1:]])
                word_vec.append(vec)
                word2ind[word]
    print (len(word2ind), 'words found for all', len(w_dct), 'words')

    return (np.array(word_vec), word2ind)


In [21]:
with open('./BioASQ-trainingDataset5b.txt', encoding='utf8') as f:
    data = json.load(f)

In [38]:
(word_vec_bio, word2ind_bio) = loadEmbedBioASQ(w_dct, vec_path='./vectors.txt', type_path='./types.txt')

15358 words found for all 16626 words


In [49]:
(word_vec_GloVe, word2ind_GloVe) = loadEmbedGloVe(w_dct, './glove.6B.300d.txt')

10540 words found for all 16626 words


In [41]:
len(word_vec[0])

200

In [43]:
word_vec.shape

(15358, 200)

In [26]:
qa_pair = getQAPair(data)
qa_pair_clean = getCleanQAPair(qa_pair)
w_dct = getWordDict(qa_pair_clean)

duplicate snippet count: 3460


In [27]:
len(w_dct)

16626

In [38]:
snippet_count.describe()

count    899.000000
mean      14.303671
std       12.157930
min        0.000000
25%        6.000000
50%       11.000000
75%       19.000000
max      121.000000
dtype: float64

In [39]:
len(qa_pair)

486

In [55]:
qa_pair[0]

['Which thyroid hormone transporter is implicated in thyroid hormone resistance syndrome?',
 ['TH monocarboxylate transporter 8 (MCT8) mutation is implicated in the TH resistance syndrome'],
 'factoid',
 ['Hemizygous MCT8 mutations in males cause severe psychomotor retardation, known as the Allan-Herndon-Dudley syndrome (AHDS), and abnormal serum TH levels. AHDS thus represents a type of TH resistance caused by a defect in cellular TH transport.',
  ' One of them, caused by mutations in the TH cell-membrane transporter MCT8, produces severe psychomotor defects.',
  ' Three syndromes, in which resistance to hormone action is respectively due to mutations in the gene encoding for thyroid hormone receptor TRβ, impaired T4 and T3 transport, and impaired conversion of T4 to T3 mediated by deiodinases.',
  'One, producing severe psychomotor defects in > 100 males from 26 families, is caused by mutations in the cell-membrane transporter of TH, MCT8;']]

In [42]:
data['questions'][0]

{'body': 'Is Hirschsprung disease a mendelian or a multifactorial disorder?',
 'concepts': ['http://www.disease-ontology.org/api/metadata/DOID:10487',
  'http://www.nlm.nih.gov/cgi/mesh/2015/MB_cgi?field=uid&exact=Find+Exact+Term&term=D006627',
  'http://www.disease-ontology.org/api/metadata/DOID:11372',
  'http://www.nlm.nih.gov/cgi/mesh/2015/MB_cgi?field=uid&exact=Find+Exact+Term&term=D020412'],
 'documents': ['http://www.ncbi.nlm.nih.gov/pubmed/15829955',
  'http://www.ncbi.nlm.nih.gov/pubmed/15617541',
  'http://www.ncbi.nlm.nih.gov/pubmed/12239580',
  'http://www.ncbi.nlm.nih.gov/pubmed/8896569',
  'http://www.ncbi.nlm.nih.gov/pubmed/6650562',
  'http://www.ncbi.nlm.nih.gov/pubmed/20598273',
  'http://www.ncbi.nlm.nih.gov/pubmed/21995290',
  'http://www.ncbi.nlm.nih.gov/pubmed/23001136'],
 'id': '55031181e9bde69634000014',
 'ideal_answer': ["Coding sequence mutations in RET, GDNF, EDNRB, EDN3, and SOX10 are involved in the development of Hirschsprung disease. The majority of these

In [ ]:
#preprocessing
with open('./types.txt') as ftype:
    for line in ftype:
        line = line.strip('\n')

## model

In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [ ]:
class BaselineModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size, embeddings, batch_size=1, num_layers=1):
        super(BaselineModel, self).__init__()
        self.batch_size = batch_size
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.word_embedding.weight = nn.Parameter(embeddings)
        
        self.qlstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)
        self.clstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)

        # The linear layer that maps from hidden state space to tag space
        self.qhidden = self.init_hidden()
        self.chidden = self.init_hidden()

        self.c2q = 
        
    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers*direction_num, minibatch_size, hidden_dim)
        batch_size = self.batch_size
        num_layers = self.num_layers
        return (autograd.Variable(torch.zeros(num_layers*2, batch_size, self.hidden_dim)),
                autograd.Variable(torch.zeros(num_layers*2, batch_size, self.hidden_dim)))

    def forward(self, question, context):
        embeds = self.word_embeddings(sentence)
        # output : (seq_len, batch, hidden_size * num_directions)
        qlstm_out, self.qhidden = self.qlstm(
            embeds.view(len(question), 1, -1), self.qhidden)
        clstm_out, self.chidden = self.clstm(
            embeds.view(len(context), 1, -1, self.chidden))
        
        attn = []
        for co in clstm_out:
            out = 
        
        
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores